In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# 1er scraping à partir des pages de test du site GameKult
Sur le site GameKult, les pages de test sont celles qui possèdent le plus d'informations sur les jeux et notamment les notes (de l'équipe de Gamekult et des joueurs).<br>
Pour faire le lien entre le nom des jeux issu du dataset du projet et les pages web de Gamekult, j'ai donc choisi la méthode suivante : <br>
1) dans un premier temps, récupération de l'ensemble des pages de tests de Gamekult<br>
2) dans chaque page de test, récupération des parties html où se trouvent les informations utiles<br>
3) cleaning des éléments scrapés pour finaliser le dataset

### 1) Récupération de l'ensemble des pages de test de Gamekult

In [2]:
# Fonction servant à récupérer les liens vers les pages de test contenus sur la page active
def getURLs(pageUrl):
    page = requests.get(pageUrl)
    parsedPage = BeautifulSoup(page.content, 'html.parser')
    sectionTest = parsedPage.find('ul', {'class':"gk__list--panel"})
    linkTest = [i['href'] for i in sectionTest.find_all('a', href=True)]
    
    if len(linkTest) > 0:
        return linkTest
    else:
        return None

In [12]:
# url de la 1ère page contenant les 1ers liens vers les pages de tests
url_datas = getURLs("https://www.gamekult.com/tests.html")

# Boucle pour récupérer les url de toutes les pages suivantes et les ajouter à la liste url_datas 
for i in range(2,1200):
    page_url = f'https://www.gamekult.com/tests.html?page={i}'
    current_page_urls = getURLs(page_url)
    if current_page_urls is not None:
        url_datas += current_page_urls
    else:
        break

# Le scraping créé des doublons que j'élimine avec le code suivant : 
url_datas = list(set(url_datas))

In [22]:
#Elimination du /test en fin d'url pour pouvoir tomber sur la page principale du jeu où se trouvent les infos

urls = []
for url in url_datas:
    url = url.split('/')
    del url[-1]
    url = '/'.join(url)
    url = "https://www.gamekult.com"+url
    urls.append(url)
    
del urls[1]
urls

['https://www.gamekult.com/jeux/box-boy-box-girl-3050880199',
 'https://www.gamekult.com/jeux/call-of-duty-modern-warfare-3-3010007926',
 'https://www.gamekult.com/jeux/mysims-racing-3010006175',
 'https://www.gamekult.com/jeux/alexandre-70067',
 'https://www.gamekult.com/jeux/lost-horizon-97573',
 'https://www.gamekult.com/jeux/ecks-vs-sever-9657',
 'https://www.gamekult.com/jeux/escape-from-bug-island-77652',
 'https://www.gamekult.com/jeux/v-rally-4-3050878121',
 'https://www.gamekult.com/jeux/madden-nfl-10-3010006493',
 'https://www.gamekult.com/jeux/evoland-2-a-slight-case-of-spacetime-continuum-disorder-3050330573',
 'https://www.gamekult.com/jeux/returnal-3050882333',
 'https://www.gamekult.com/jeux/singularity-3010005652',
 'https://www.gamekult.com/jeux/pathfinder-wrath-of-the-righteous-3050881573',
 'https://www.gamekult.com/jeux/skate-2-3010005345',
 'https://www.gamekult.com/jeux/grand-theft-auto-iv-the-lost-and-damned-3010006943',
 'https://www.gamekult.com/jeux/dragon-que

### 2) Récupération des éléments html dans chaque page de test

In [23]:
#Fonction pour récupérer les infos nécessaires dans une page de test
#Cette fonction prend en argument une url et renvoie un dictionnaire contenant les informations scrapées

def extract_data(url):

    # Scrap : nom du jeu
    name = parsedPage.find('a', {'class':'js-title'})
    if name is not None:
        name = name.text.strip()
    else:
        name = None
    
    # Scrap : note joueurs
    note_joueur = parsedPage.select(".gk__user-note__md span")
    note_joueur = str(note_joueur)
    if len(parsedPage.select(".gk__user-note__md span")) != 0:
        note_joueur = note_joueur.split('<')
        note_joueur_split = []
        for i in note_joueur:
            note_joueur_split += i.split('>')
        if len(note_joueur_split) < 3 :
            note_joueur_data = None
        else:
            note_joueur_data = note_joueur_split[2].strip()
    else:
        note_joueur_data = None
     
    
    # Scrap : nb avis
    nb_avis = parsedPage.select(".us__reviews-h__opinion-count a")
    nb_avis = str(nb_avis)
    if len(parsedPage.select(".us__reviews-h__opinion-count a")) != 0:
        nb_avis = nb_avis.split('(')
        nb_avis = nb_avis[1].split(')')
        nb_avis = nb_avis[0].split(' ')
        nb_avis = nb_avis[2]
    else:
        nb_avis = None
    
    
    # Scrap : note GK
    note_GK = parsedPage.select(".gk__staff-note__inset span")
    note_GK = str(note_GK)
    note_GK = note_GK.split('<')
    note_GK_split = []
    for i in note_GK:
        note_GK_split += i.split('>')
    if len(note_GK_split) < 3 :
        note_GK_data = None
    else:
        note_GK_data = note_GK_split[2].strip()
    
    
    # Scrap zone infos
    zt = parsedPage.find_all('th', {'class':'gk__data-table__item'})
    zc = parsedPage.find_all('td', {'class':'gk__data-table__value gk__text-left'})
    list_info_titre = []
    list_info_contenu = []
    for th in zt:
        list_info_titre.append(th.text)
    for td in zc:
        list_info_contenu.append(td.text)
    
    
    
    data = {
        'Name' : name,
        'Note_joueurs' : note_joueur_data,
        'Nb_avis' : nb_avis,
        'Note_GK' : note_GK_data,
        'Zone_titre' : list_info_titre,
        'Zone_contenu' : list_info_contenu
        }
    
    return data

In [28]:
#Création du dataframe 
data_full = []
for url in urls:
    scrap = requests.get(url)
    parsedPage = BeautifulSoup(scrap.content, 'html.parser')
    data_full.append(extract_data(parsedPage))
    
data_pd = pd.DataFrame.from_dict(data_full)

### 3) Cleaning des éléments html scrapé pour finaliser le dataset

In [29]:
df = data_pd

In [30]:
#Cleaning de zone_titre et zone_contenu où se trouvent les infos concernant le distributeur, le développeur, l'éditeur, la franchise de chaque jeu

zone_dico = []

for i in range(0,df.shape[0]):
    a = df['Zone_titre'][i][:6]
    for x in range(0, len(a)):
        a[x] = a[x].strip("\n\n")
        a[x] = a[x].replace('\\n\\n                                        ', '')
        a[x] = a[x].replace('\n', '')
        a[x] = a[x].replace('\\n                                    \\n', '')
        a[x] = a[x].replace('\\n', '')
        a[x] = a[x].replace('  ', '')
    b = df['Zone_contenu'][i][:6]
    for x in range(0, len(b)):
        b[x] = b[x].strip("\n\n")
        b[x] = b[x].replace('\\n\\n                                        ', '')
        b[x] = b[x].replace('\n', '')
        b[x] = b[x].replace('\\n                                    \\n', '')
        b[x] = b[x].replace('\\n', '')
        b[x] = b[x].replace('  ', '')

    dict_test = {
        'Name' : df['Name'][i],
        'Distributeur' : None,
        'Développeur' : None, 
        'Éditeur' : None, 
        'Franchise' : None, 
    }

    dict_zone = {}

    for x,y in zip(a,b):
        if x in dict_test.keys():
            dict_a = {x:y}
            dict_zone = dict(dict_zone, **dict_a)

    dict_test = dict(dict_test, **dict_zone)

    zone_dico.append(dict_test)
    

dico_df = pd.DataFrame.from_dict(zone_dico)

In [31]:
#Merge des 2 sous-datasets pour obtenir le dataset final issu du 1er scraping
df_GK = df.drop(['Zone_titre', 'Zone_contenu'], axis=1)
df_GK = df_GK.merge(right=dico_df,on='Name',how='left')

In [32]:
df_GK.to_csv('../data/01_GameSpy_Scraping_gk_01.csv')